import numpy as np
import math
from itertools import combinations

In [10]:
var_counter = 0

def generate_n_tuple(n, i, j, n_vert):
    # Generate an n-tuple of distinct vertices from 1 to n_vert, excluding i and j
    return list(combinations([k for k in range(1, n_vert + 1) if k != i and k != j], n))


def write_cnf_file(n_vert):
    global var_counter

    file_name = str(n_vert) + "_graph.txt"

    n_var = math.comb(n_vert, 2) * (n_vert - 1 + math.comb(n_vert - 2, 2))
    n_clause = math.comb(n_vert, 2) * (2 + 3 * (n_vert - 2) + 6 * math.comb(n_vert - 2, 2) + math.comb(n_vert - 2, 3))
    
    var_counter = math.comb(n_vert, 2)
    
    with open(file_name, "w") as f:
        f.write(f'p cnf {n_var} {n_clause}\n')

        for i, j in combinations(range(1, n_vert + 1), 2):

            triangle_existence(i, j, n_vert, f)
            triangle_uniqueness(i, j, n_vert, f)
            quadrilateral_uniqueness(i, j, n_vert, f)
            quadrilateral_existence(i, j, n_vert, f)

            
def triangle_existence(i, j, n_vert, f):
    global var_counter

    distinct_vertices = generate_n_tuple(1, i, j, n_vert)

    r = range(1 + var_counter, len(distinct_vertices) + 1 + var_counter)

    s = " ".join(str(i) for i in r)

    ij = pair_index(i, j, n_vert)
    
    f.write(f"-{ij} {s} 0\n")

    for k in distinct_vertices:
        var_counter += 1
        ik = pair_index(i, k, n_vert)
        jk = pair_index(j, k, n_vert)
        f.write(f"-{ij} {-var_counter} {ik} 0\n")
        f.write(f"-{ij} {-var_counter} {jk} 0\n")
        f.write(f"-{ij} {var_counter} -{ik} -{jk} 0\n")

        
def quadrilateral_existence(i, j, n_vert, f):
    global var_counter
    
    pairs = generate_n_tuple(2, i, j, n_vert)

    r = range(1 + var_counter, len(pairs) + 1 + var_counter)

    s = " ".join(str(i) for i in r)

    ij = pair_index(i, j, n_vert)
        
    f.write(f"{ij} {s} 0\n")

    for a, b in pairs:
        var_counter += 1
        ia = pair_index(i, a, n_vert)
        ib = pair_index(i, b, n_vert)
        ja = pair_index(j, a, n_vert)
        jb = pair_index(j, b, n_vert)
        f.write(f"{ij} {-var_counter} {ia} 0\n")
        f.write(f"{ij} {-var_counter} {ib} 0\n")
        f.write(f"{ij} {-var_counter} {ja} 0\n")
        f.write(f"{ij} {-var_counter} {jb} 0\n")
        f.write(f"{ij} {var_counter} -{ia} -{ib} -{ja} -{jb} 0\n")

    
def triangle_uniqueness(i, j, n_vert, f):
    
    pairs = generate_n_tuple(2, i, j, n_vert)

    for a, b in pairs:
        result = " ".join([f"-{pair_index(k1, k2, n_vert)}" for k1, k2 in [(i, j), (i, a), (i, b), (j, a), (j, b)]])
        f.write(f"{result} 0\n")
    
    
def quadrilateral_uniqueness(i, j, n_vert, f):
    
    triples = generate_n_tuple(3, i, j, n_vert)

    ij = pair_index(i, j, n_vert)
    
    for a, b, c in triples:
        result = " ".join([ij] + [f"-{pair_index(k1, k2, n_vert)}" for k1, k2 in [(i, a), (i, b), (i, c), (j, a), (j, b), (j, c)]])
        f.write(f"{result} 0\n")
        
        
def pair_index(i, j, n_vert):
    i, j = sorted([i, j])
    i -= 1
    j -= 1
    return str((i * (n_vert - 1) - i * (i - 1) // 2 + j - i - 1) + 1)

In [8]:
for i in range(3, 15):
    write_cnf_file(i)